# Categorizando mascotas con redes neuronales

### Introducción

En este cuaderno Jupyter aprenderás a clasificar imágenes de mascotas de forma automática, utilizando la potencia de las redes neuronales convolucionales, la técnica puntera que ha supuesto el _boom_ del Deep Learning.

### Carga de datos
Habremos descargado el conjunto de datos **Oxford pets** del URL <https://www.robots.ox.ac.uk/~vgg/data/pets/> y extraído las imágenes a una carpeta `images`. La siguiente celda organiza los archivos en dos clases (perros y gatos) y en dos subconjuntos (entrenamiento y test) para facilitar las tareas posteriores:

In [ ]:
# Descargar e instalamos la libreria TensorFlow
%pip install tensorflow

In [1]:
# Importamos las librerías necesarias para la descarga y extracción de los archivos
# La librería os nos permite interactuar con el sistema operativo y la librería tarfile nos permite trabajar con archivos comprimidos
import os
import tarfile
# La librería urllib.request nos permite descargar archivos de la web
# requests nos permite hacer peticiones a servidores web y trabajar con sus respuestas
import urllib.request

# Definir la URL del archivo comprimido
url1 = "https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz"

url2 = "https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz"

# Definir el directorio para almacenar el contenido extraído
directory1 = "images"
directory2 = "annotations"

# Crea un directorio para almacenar los archivos extraídos
os.makedirs(directory1, exist_ok=True)
os.makedirs(directory2, exist_ok=True)

# Descargar el archivo comprimido
filename1, _ = urllib.request.urlretrieve(url1)
filename2, _ = urllib.request.urlretrieve(url2)

# Extraer el contenido del archivo comprimido
with tarfile.open(filename1, "r:gz") as tar:
  tar.extractall(directory1)

# Extraer el contenido del archivo comprimido
with tarfile.open(filename2, "r:gz") as tar:
  tar.extractall(directory2)

# Eliminar el archivo comprimido
os.remove(filename1)
os.remove(filename2)


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importamos las librerías necesarias para el desarrollo del modelo de machine learning
import os

# Definimos las rutas de las carpetas que contienen las imágenes y las anotaciones
images_path = "images/images"
# Anotaciones de las imágenes
annotations_path = "annotations/annotations"

# open() abre el archivo que se le pasa como argumento y readlines() lee todas las líneas del archivo y las devuelve como una lista
trainval = open(os.path.join(annotations_path, "trainval.txt")).readlines()
test = open(os.path.join(annotations_path, "test.txt")).readlines()

# Creamos las carpetas necesarias para clasificar las imágenes en función de si son de gatos o de perros
os.makedirs(os.path.join(images_path, "train", "cats"), exist_ok=True)
os.makedirs(os.path.join(images_path, "train", "dogs"), exist_ok=True)
os.makedirs(os.path.join(images_path, "test", "cats"), exist_ok=True)
os.makedirs(os.path.join(images_path, "test", "dogs"), exist_ok=True)

# Definimos una función que clasifica las imágenes en función de si son de gatos o de perros
def classify_image(line, subset):
# basename : Devuelve el último componente de la ruta que se le pasa como argumento
    basename = line.split(" ")[0]
# species : Devuelve el tercer componente de la línea que se le pasa como argumento
    species = line.split(" ")[2]
# subfolder : Es igual a "cats" si species es igual a "1" y "dogs" en caso contrario    
    subfolder = "cats" if species == "1" else "dogs"
# oldpath : Es igual a la ruta de la imagen    
    oldpath = os.path.join(images_path, f"{basename}.jpg")
# newpath : Es igual a la ruta de la imagen en función de si es de gato o de perro    
    newpath = os.path.join(images_path, subset, subfolder, f"{basename}.jpg")
# si oldpath : Es un archivo, se renombra a newpath
    if os.path.isfile(oldpath):
        # os.rename() renombra el archivo que se le pasa como primer argumento con el nombre que se le pasa como segundo argumento
        os.rename(oldpath, newpath)

# Clasificamos las imágenes en función de si son de gatos o de perros
for line in trainval:
    classify_image(line, "train")

# Clasificamos las imágenes en función de si son de gatos o de perros
for line in test:
    classify_image(line, "test")

In [3]:
# Importamos las librerías necesarias para el desarrollo del modelo de machine learning
# La librería tensorflow.keras.preprocessing.image contiene la clase ImageDataGenerator 
# que permite generar lotes de tensores con datos de imagen en tiempo real
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Creamos un generador de datos de imagen para el conjunto de entrenamiento
generador_entrenamiento = ImageDataGenerator()
# flow_from_directory() genera lotes de tensores con datos de imagen en tiempo real
datos_entrenamiento = generador_entrenamiento.flow_from_directory("images/train")
# ImageDataGenerator() genera lotes de tensores con datos de imagen en tiempo real
generador_test = ImageDataGenerator()
# flow_from_directory() genera lotes de tensores con datos de imagen en tiempo real
datos_test = generador_test.flow_from_directory("images/test", class_mode=None)
# next(datos_test) devuelve el siguiente lote de tensores con datos de imagen
algunas_imagenes = next(datos_test)


Found 3680 images belonging to 2 classes.
Found 3669 images belonging to 2 classes.


### Visualización

Podemos visualizar algún ejemplo de imagen a continuación:

In [ ]:
# matplotlib.pyplot es una colección de funciones que hacen que matplotlib funcione como MATLAB
from matplotlib import pyplot as plt

# Mostramos algunas imágenes
plt.imshow(algunas_imagenes[0]/255.)
# plt.axis('off') elimina los ejes de la imagen para que no se muestren en el gráfico
plt.axis('off')
# plt.show() muestra el gráfico
plt.show()
# plt.imshow() muestra la imagen que se le pasa como argumento en el gráfico para que se muestre en el gráfico la siguiente imagen
plt.imshow(algunas_imagenes[1]/255.)
# plt.axis('off') elimina los ejes de la imagen para que no se muestren en el gráfico para que no se muestren en el gráfico
plt.axis('off')
# plt.show() muestra el gráfico
plt.show()

### Carga del modelo

Nuestro objetivo será crear un modelo capaz de responder a la pregunta "¿Corresponde esta imagen a un gato o a un perro?". En lugar de diseñar una nueva red neuronal desde cero, podemos cargar una red ya construida y, mejor aún, los parámetros optimizados para el conjunto de datos `Imagenet` de todo tipo de imágenes, de forma que nuestra red viene ya "preparada" para reconocer imágenes y no partimos de cero al entrenar. Esta estrategia se conoce como _transfer learning_.

Importaremos la red InceptionV3 desde la biblioteca de modelos ya entrenados de Tensorflow. Esta red se basa en un componente llamado "bloque Inception": encadena varios de estos bloques para extraer información de la imagen.

In [5]:
# Importamos las librerías necesarias para el desarrollo del modelo de machine learning
# keras es una API de redes neuronales de alto nivel escrita en Python que se utiliza para el desarrollo de modelos de machine learning y deep learning su función principal es la de ser una interfaz de alto nivel para la biblioteca TensorFlow
# applications contiene una serie de modelos pre-entrenados que se pueden utilizar para el desarrollo de modelos de machine learning y deep learning
from tensorflow.keras import applications
# InceptionV3 es una red neuronal convolucional que se utiliza para el desarrollo de modelos de machine learning y deep learning
# Su función principal es la de ser una interfaz de alto nivel para la biblioteca TensorFlow
inception = applications.InceptionV3(include_top=False, input_shape=(256, 256, 3))

### Ajustes del modelo

En la siguiente celda añadimos a la red InceptionV3 un par de capas que nos permiten obtener una predicción a partir de la información que haya inferido de la imagen.

In [6]:
# Importamos las librerías necesarias para el desarrollo del modelo de machine learning
# keras es una API de redes neuronales de alto nivel escrita en Python que se utiliza 
# para el desarrollo de modelos de machine learning y deep learning su función principal 
# es la de ser una interfaz de alto nivel para la biblioteca TensorFlow

# tensorflow.keras.layers contiene las clases Flatten y Dense
# La clase Flatten que se utiliza para aplanar los datos de entrada en una red neuronal  
# La clase Dense que se utiliza para crear capas densas en una red neuronal
from tensorflow.keras.layers import Flatten, Dense
# tensorflow.keras.models contiene la clase Sequential que se utiliza para crear modelos de machine learning 
from tensorflow.keras.models import Sequential

# Creamos la variable predictor que es igual a un modelo secuencial que contiene una capa Flatten, una capa Dense con 128 neuronas y una función de activación relu y una capa Dense con 2 neuronas y una función de activación softmax
predictor = Sequential([
    Flatten(), 
    # Dense() crea una capa densa en una red neuronal de 128 neuronas y una función de activación recibe como argumento la función de activación relu 
    # "relu" : es una función de activación que se utiliza en las redes neuronales
    Dense(128, activation="relu"), 
    # Dense() tiene 2 neuronas y una función de activación softmax 
    # "softmax" : se utiliza en las redes neuronales para clasificación y su función principal es la de convertir los valores de las neuronas en probabilidades
    Dense(2, activation="softmax")
])
# Sequential() crea un modelo secuencial que recibe como argumento una lista de capas
# inception : es un modelo pre-entrenado que se utiliza como capa de extracción de características
# predictor : es un modelo secuencial que se utiliza como clasificador
modelo = Sequential([inception, predictor])
# compile() compila el modelo que se le pasa como argumento
# funcion optimizer : es el optimizador que se utiliza para el entrenamiento del modelo 
# El parametro "adam" es un optimizador que se utiliza en las redes neuronales
# funcion loss : es la función de pérdida que se utiliza para el entrenamiento del modelo 
# El parametro "categorical_crossentropy" es una función de pérdida que se utiliza en las redes neuronales
modelo.compile(optimizer="adam", loss="categorical_crossentropy")

### Entrenamiento

Una vez creado el modelo que ya tiene la estructura final para responder preguntas de "sí/no", ajustamos sus parámetros (que inicialmente son aleatorios) al conjunto de imágenes que vamos a utilizar para entrenar:

In [9]:
# modelo.fit(datos_entrenamiento, epochs=50)
# fit() entrena el modelo que se le pasa como argumento 
# datos_entrenamiento : es el conjunto de datos de entrenamiento que se utiliza para el entrenamiento del modelo
# epochs : es el número de épocas que se utiliza para el entrenamiento del modelo
modelo.fit(datos_entrenamiento, epochs=10)

Epoch 1/10
115/115 [==============================] - 425s 4s/step - loss: 0.0491
Epoch 2/10
115/115 [==============================] - 424s 4s/step - loss: 0.0187
Epoch 3/10
115/115 [==============================] - 426s 4s/step - loss: 0.0104
Epoch 4/10
115/115 [==============================] - 428s 4s/step - loss: 0.0279
Epoch 5/10
115/115 [==============================] - 428s 4s/step - loss: 0.0126
Epoch 6/10
115/115 [==============================] - 444s 4s/step - loss: 0.0294
Epoch 7/10
115/115 [==============================] - 430s 4s/step - loss: 0.0199
Epoch 8/10
115/115 [==============================] - 441s 4s/step - loss: 0.0417
Epoch 9/10
115/115 [==============================] - 459s 4s/step - loss: 0.0219
Epoch 10/10
115/115 [==============================] - 446s 4s/step - loss: 0.0233


### Predicción

Nuestro modelo ya está listo. En la siguiente celda tomamos algunas imágenes del subconjunto de test (imágenes que nunca han sido vistas por la red neuronal) y comprobamos cuáles son las predicciones del modelo: ¿acertará todos los perros y gatos?

In [10]:
# next(datos_test) devuelve el siguiente lote de tensores con datos de imagen
lote_test = next(datos_test)

# modelo.predict(lote_test) devuelve las predicciones del modelo que se le pasa como argumento
probs = modelo.predict(lote_test)
# La librería numpy es una biblioteca de funciones matemáticas de alto nivel que se utiliza para el desarrollo de modelos de machine learning y deep learning
import numpy as np
# np.argmax(probs, -1) devuelve el índice del valor máximo de las predicciones
# probs : es el conjunto de predicciones que se le pasa como argumento
# -1 : es el eje en el que se busca el valor máximo
clase = np.argmax(probs, -1)

1/1 [==============================] - 3s 3s/step


In [ ]:
# Mostramos las predicciones para las imágenes del lote de test
mostrar_imagenes = 10

# Mostramos las predicciones para las imágenes del lote de test mediante un bucle for que recorre el rango de mostrar_imagenes 
for i in range(mostrar_imagenes):
# plt.imshow() muestra la imagen que se le pasa como argumento en el gráfico para que se muestre en el gráfico la siguiente imagen 
    plt.imshow(lote_test[i]/255.)
# plt.axis('off') elimina los ejes de la imagen para que no se muestren en el gráfico
    plt.axis('off')
# plt.show() muestra el gráfico    
    plt.show()
# print() muestra el texto que se le pasa como argumento y el resultado de la predicción que se le pasa como argumento 
    print("Predicción:", "perro" if clase[i] else "gato")